In [1]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')


In [2]:
spark = (
    SparkSession
    .builder
    .appName("landing_to_bronze")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .config("spark.driver.memory", "24G")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ca8ab36c-0696-472b-971f-390a79ccfada;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 715ms :: artifacts dl 16ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from c

In [3]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [4]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType, StructType
from dateutil import parser
from delta.tables import *

In [5]:
bronze_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/ecommerce/bnsvher602/linx_db/dbo/hr_omni_pedido/'
silver_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/ecommerce/bnsvher602/linx_db/dbo/hr_omni_pedido/'
silver_path_restricted = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/ecommerce/bnsvher602/linx_db/dbo/hr_omni_pedido_restrito/'

In [6]:
# Reading last silver data
try:
    df_silver = (
        spark
        .read
        .format("delta")
        .load(silver_path)
    )

    list_result = (
        df_silver
        .agg({"timestamp_kafka": "max"}).collect()
    )

    last_date = list_result[0].asDict()['max(timestamp_kafka)']

except:
    last_date = parser.parse("01-01-2021 12:00AM")

In [7]:
print(last_date)

2021-01-01 00:00:00


In [8]:
df_table = (
    spark
    .read
    .format("delta")
    .load(bronze_path)
    .where((f.col("year")>=last_date.year)&(f.col("month")>=last_date.month)&(f.col("day")>=last_date.day))
    .where(f.col("timestamp_kafka")>last_date)
)

In [9]:
df_table_restricted = (
    df_table
    .select(f.col('PEDIDO'), f.col('CODIGO_CLIENTE'), f.col('__op'))
    .withColumn("CODIGO_CLIENTE_HASH", f.sha2(f.col('CODIGO_CLIENTE'), 256))    
)

In [10]:
df_table_hash = (
    df_table
    .withColumn("CODIGO_CLIENTE", f.sha2(f.col('CODIGO_CLIENTE'), 256))
)

In [11]:
# Snapshot hash
if df_table_hash.where((f.col('__op') == 'r')|(f.col('__op')=='c')).count() > 0:
    df_table_r = df_table_hash.where(f.col('__op') == 'r')
    (
        df_table_r
        .write
        .format("delta")
        .mode("append")
        .option("checkpointLocation", silver_path + "checkpoint")
        .save(silver_path)
    )
    
# Insert and Update
if df_table_hash.where(f.col('__op') != 'r').count() > 0:
    silver_table = DeltaTable.forPath(spark, silver_path)
    df_table_u = df_table_hash.where(f.col('__op') != 'r')
    df_table_u_unique = df_table_u.orderBy('offset', ascending = False).dropDuplicates(["PEDIDO"])
    (
        silver_table.alias('silver')
        .merge(df_table_u_unique.alias('updates'),'silver.PEDIDO = updates.PEDIDO')
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )
    
# Delete
if df_table_hash.where(f.col('__op') == 'd').count() > 0:
    silver_table = DeltaTable.forPath(spark, silver_path)
    silver_table.delete(col('__op') == 'd')
    

In [ ]:
# Snapshot restrited
if df_table_restricted.where((f.col('__op') == 'r')|(f.col('__op')=='c')).count() > 0:
    df_table_restricted_r = df_table_restricted.where(f.col('__op') == 'r')
    (
        df_table_restricted_r
        .write
        .format("delta")
        .mode("append")
        .option("checkpointLocation", silver_path_restricted + "checkpoint")
        .save(silver_path_restricted)
    )
    
# Insert and Update
if df_table_restricted.where(f.col('__op') != 'r').count() > 0:
    silver_table_restricted = DeltaTable.forPath(spark, silver_path_restricted)
    df_table_restricted_u = df_table_restricted.where(f.col('__op') != 'r')
    df_table_restricted_u_unique = df_table_u.orderBy('offset', ascending = False).dropDuplicates(["PEDIDO"])
    (
        silver_table_restricted.alias('silver')
        .merge(df_table_restricted_u_unique.alias('updates'),'silver.PEDIDO = updates.PEDIDO')
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )
    
# Delete
if df_table_restricted.where(f.col('__op') == 'd').count() > 0:
    silver_table_restricted = DeltaTable.forPath(spark, silver_path_restricted)
    silver_table_restricted.delete(col('__op') == 'd')
    

In [ ]:
##################################
### Verificar está funcionando ###
##################################

In [13]:
df_silver_hash_t = (
    spark
    .read
    .format("delta")
    .load(silver_path)
)

df_silver_restricted_t = (
    spark
    .read
    .format("delta")
    .load(silver_path_restricted)
)

In [14]:
df_silver_hash_t.join(df_silver_restricted_t,df_silver_hash_t.PEDIDO == df_silver_restricted_t.PEDIDO,"inner").limit(100).toPandas()

offset         timestamp_kafka  year  month  day  hour CODIGO_SITE  \
0      -19 2022-03-10 17:28:55.872  2022      3   10    17         001   
1      -36 2022-03-10 17:28:55.961  2022      3   10    17         001   
2     -273 2022-03-10 17:28:56.569  2022      3   10    17         001   
3     -490 2022-03-10 17:28:56.862  2022      3   10    17         001   
4     -627 2022-03-10 17:28:56.987  2022      3   10    17         001   
..     ...                     ...   ...    ...  ...   ...         ...   
95  -11695 2022-03-10 17:29:04.884  2022      3   10    17         001   
96  -11974 2022-03-10 17:29:04.961  2022      3   10    17         001   
97  -11933 2022-03-10 17:29:04.969  2022      3   10    17         001   
98  -12080 2022-03-10 17:29:04.981  2022      3   10    17         001   
99  -12203 2022-03-10 17:29:04.991  2022      3   10    17         001   

        PEDIDO           DATA  TIPO_PEDIDO  ... DEVE_FATURAR  \
0   1000002244  1640131200000            1  ...         None   
1   1000006796  1640131200000            1  ...         None   
2   1000054027  1640131200000            1  ...         None   
3   1000107426  1640131200000            1  ...         None   
4   1000137973  1640131200000            1  ...         None   
..         ...            ...          ...  ...          ...   
95  1008027089  1641340800000            1  ...         None   
96  1008168081  1641340800000            1  ...         None   
97  1008229269  1641427200000            1  ...         None   
98  1008424158  1641427200000            1  ...         None   
99  1008500451  1641427200000            1  ...         None   

   PEDIDO_DIFERENTE_RESERVA  __op         __table  __source_ts_ms  __deleted  \
0                      True     r  HR_OMNI_PEDIDO   1646933333780      false   
1                      True     r  HR_OMNI_PEDIDO   1646933333861      false   
2                      True     r  HR_OMNI_PEDIDO   1646933334177      false   
3                      True     r  HR_OMNI_PEDIDO   1646933334285      false   
4                      None     r  HR_OMNI_PEDIDO   1646933334365      false   
..                      ...   ...             ...             ...        ...   
95                     True     r  HR_OMNI_PEDIDO   1646933341876      false   
96                     True     r  HR_OMNI_PEDIDO   1646933341981      false   
97                     True     r  HR_OMNI_PEDIDO   1646933342057      false   
98                     True     r  HR_OMNI_PEDIDO   1646933342156      false   
99                     True     r  HR_OMNI_PEDIDO   1646933342185      false   

        PEDIDO CODIGO_CLIENTE  __op  \
0   1000002244  8998316802052     r   
1   1000006796  8995072475140     r   
2   1000054027  9242919567364     r   
3   1000107426  9210949664772     r   
4   1000137973  9011515555844     r   
..         ...            ...   ...   
95  1008027089  8926314168324     r   
96  1008168081  9247966232580     r   
97  1008229269  9247868321796     r   
98  1008424158  9245054730244     r   
99  1008500451  9248324943876     r   

                                  CODIGO_CLIENTE_HASH  
0   fa5d8cae9919ffb099aad016c3a3577dba6b28a7aabbc8...  
1   dbd380acfa577003d156385b6aa19871f5e23354345a8c...  
2   858d379f8929e29f74dd7abc353f133b753f082557e618...  
3   579aa8f74648539b7d6321bd9a712aa615abbb9f27e062...  
4   712f94d48ce06f3f00c87612e1b46a26561a69a1ed1978...  
..                                                ...  
95  4d6dc3515ec2075a3b9666c1f219d4c2229b460c092a3d...  
96  cf928c9c4ade4637f8bde879778235f6e624753d12d166...  
97  70db21fd80ecb9589eac984f7223317f20c61611286717...  
98  12cf9bf4232ee9657373b5f38829b1617dad5138f173a6...  
99  df449aa4f61051d66f355007b035b46373f651120f9468...  

[100 rows x 43 columns]

In [26]:
df_pd = df_silver_teste.toPandas()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series


In [27]:
df_pd

offset         timestamp_kafka  year  month  day  hour       PRODUTO  \
0      -24161 2022-03-09 13:45:47.668  2022      3    9    13  012N           
1       -7625 2022-03-09 13:45:04.422  2022      3    9    13  014V           
2       -7721 2022-03-09 13:45:04.425  2022      3    9    13  015W           
3      -36316 2022-03-09 13:46:14.155  2022      3    9    13  01YP           
4      -24283 2022-03-09 13:45:47.681  2022      3    9    13  0295           
...       ...                     ...   ...    ...  ...   ...           ...   
97900   -7864 2022-03-09 13:45:06.460  2022      3    9    13  ZMBD           
97901  -42646 2022-03-09 13:46:33.061  2022      3    9    13  ZU6N           
97902  -48612 2022-03-09 13:47:00.239  2022      3    9    13  ZU99           
97903   -9941 2022-03-09 13:45:13.131  2022      3    9    13  ZUJA           
97904  -39683 2022-03-09 13:46:26.686  2022      3    9    13  ZUL9           

      CODIGO_PRECO MATERIAL PERIODO_PCP  ... HR_B2C_CATEGORIA  ID_CEST_NCM  \
0                      None        None  ...             None          NaN   
1                      None        None  ...             None          NaN   
2                      None        None  ...             None          NaN   
3                      None        None  ...             None          NaN   
4                      None        None  ...             None          NaN   
...            ...      ...         ...  ...              ...          ...   
97900                  None        None  ...             None          NaN   
97901                  None        None  ...             None          NaN   
97902                  None        None  ...             None          NaN   
97903                  None        None  ...             None          NaN   
97904                  None        None  ...             None          NaN   

      HR_INDICA_BRINDE HR_PARCEIRO HR_ESTRATEGIA INFO_MODELO __op   __table  \
0                 None        None          None        None    r  PRODUTOS   
1                 None        None          None        None    r  PRODUTOS   
2                 None        None          None        None    r  PRODUTOS   
3                 None        None          None        None    r  PRODUTOS   
4                 None        None          None        None    r  PRODUTOS   
...                ...         ...           ...         ...  ...       ...   
97900             None        None          None        None    r  PRODUTOS   
97901             None        None          None        None    r  PRODUTOS   
97902             None        None          None        None    r  PRODUTOS   
97903             None        None          None        None    r  PRODUTOS   
97904             None        None          None        None    r  PRODUTOS   

      __source_ts_ms __deleted  
0      1646833546855     false  
1      1646833504018     false  
2      1646833504065     false  
3      1646833573323     false  
4      1646833547233     false  
...              ...       ...  
97900  1646833504834     false  
97901  1646833592148     false  
97902  1646833618534     false  
97903  1646833512635     false  
97904  1646833584853     false  

[97905 rows x 165 columns]

In [30]:
df_pd[df_pd["__op"] != "r"]

offset         timestamp_kafka  year  month  day  hour       PRODUTO  \
34      52663 2022-03-16 04:41:02.566  2022      3   16     4  06AK           
130     27191 2022-03-11 02:46:38.729  2022      3   11     2  4A8Y           
147     30130 2022-03-11 04:40:54.406  2022      3   11     4  4E25           
160     50758 2022-03-15 17:50:28.414  2022      3   15    17  4FPQ           
174     78439 2022-03-17 05:35:15.868  2022      3   17     5  54NA           
...       ...                     ...   ...    ...  ...   ...           ...   
97735   28635 2022-03-11 04:40:54.389  2022      3   11     4  H98D           
97736      59 2022-03-08 15:27:34.147  2022      3    8    15  H9CQ           
97757   77726 2022-03-17 05:44:28.417  2022      3   17     5  KFKN           
97794   64643 2022-03-16 17:50:57.216  2022      3   16    17  M06H           
97831   28760 2022-03-11 04:40:54.404  2022      3   11     4  UBL0           

      CODIGO_PRECO MATERIAL PERIODO_PCP  ... HR_B2C_CATEGORIA  ID_CEST_NCM  \
34                     None        None  ...             None          NaN   
130                    None        None  ...             None          NaN   
147                    None        None  ...             None          NaN   
160                    None        None  ...             None          NaN   
174                    None        None  ...         INFANTIL          NaN   
...            ...      ...         ...  ...              ...          ...   
97735                  None        None  ...             None          NaN   
97736                  None        None  ...             None          NaN   
97757                  None        None  ...             None          NaN   
97794                  None        None  ...             None          NaN   
97831                  None        None  ...             None          NaN   

      HR_INDICA_BRINDE HR_PARCEIRO HR_ESTRATEGIA INFO_MODELO __op   __table  \
34                None        None          None        None    u  PRODUTOS   
130               None        None          None        None    u  PRODUTOS   
147               None        None          None        None    u  PRODUTOS   
160               None        None          None        None    u  PRODUTOS   
174               None        None          None        None    u  PRODUTOS   
...                ...         ...           ...         ...  ...       ...   
97735             None        None          None        None    u  PRODUTOS   
97736             None        None          None        None    u  PRODUTOS   
97757             None        None          None        None    u  PRODUTOS   
97794             None        None          None        None    u  PRODUTOS   
97831             None        None          None        None    u  PRODUTOS   

      __source_ts_ms __deleted  
34     1647405657487     false  
130    1646966792393     false  
147    1646973646740     false  
160    1647366624380     false  
174    1647495310903     false  
...              ...       ...  
97735  1646973646740     false  
97736  1646753247243     false  
97757  1647495865523     false  
97794  1647453051560     false  
97831  1646973646740     false  

[1975 rows x 165 columns]